In [1]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df1 = pd.read_excel('/content/drive/MyDrive/음성인식/구어체1.xlsx')
df1

,SID,원문,번역문
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2,씨티은행에서 일하세요?,Do you work at a City bank?
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
...,...,...,...
199995,199996,나는 먼저 청소기로 바닥을 밀었어요.,"First of all, I vacuumed the floor."
199996,199997,나는 먼저 팀 과제를 하고 놀러 갔어요.,I did the team assignment first and went out t...
199997,199998,나는 비 같은 멋진 연예인을 좋아해요.,I like cool entertainer like Rain.
199998,199999,나는 멋진 자연 경치를 보고 눈물을 흘렸어.,I cried seeing the amazing scenery.


In [4]:
df2 = pd.read_excel('/content/drive/MyDrive/음성인식/구어체2.xlsx')
df2

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
199995,399996.0,당신은 곧 당신의 조카를 볼 수 있어요.,You can see your nephew soon.
199996,399997.0,나는 당신이 내게 공유한 음악을 들었어요.,I've listened to the music that you shared wit...
199997,399998.0,그 서류는 당신이 공유해 준 것과 달라요.,The document is different from what you shared.
199998,399999.0,당신이 잡지에 광고 낸 것을 봤어요.,I saw your advertisement in the magazine.


In [5]:
df = pd.concat([df1,df2])
df

,SID,원문,번역문
0,1.0,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2.0,씨티은행에서 일하세요?,Do you work at a City bank?
2,3.0,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4.0,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5.0,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
...,...,...,...
199995,399996.0,당신은 곧 당신의 조카를 볼 수 있어요.,You can see your nephew soon.
199996,399997.0,나는 당신이 내게 공유한 음악을 들었어요.,I've listened to the music that you shared wit...
199997,399998.0,그 서류는 당신이 공유해 준 것과 달라요.,The document is different from what you shared.
199998,399999.0,당신이 잡지에 광고 낸 것을 봤어요.,I saw your advertisement in the magazine.


In [6]:
kor = df['원문']
eng = df['번역문']

## Configuration

In [7]:
batch_size = 64 
epochs = 100
hidden_units = 128
embedding_dim = 64
num_samples = 35000

## Prepare the data


데이터셋을 다운로드한 후에 데이터를 준비하고자 다음의 단계를 수행합니다.

1. 각 문장에 start와 end 토큰을 추가합니다.
2. 특정 문자를 제거함으로써 문장을 정리합니다.
3. 단어 인덱스와 아이디(ID) 인덱스를 생성합니다. (단어 → 아이디(ID), 아이디(ID) → 단어로 매핑된 딕셔너리).
4. 각 문장을 입력층의 최대 길이만큼 패딩(padding)을 수행합니다.

In [8]:
def preprocess_sentence(sent):
  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^가-힣a-zA-Z.!?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [9]:
encoder_input, decoder_input, decoder_target = [], [], []

In [10]:
print(len(eng))
print(len(kor))

400000
400000


In [11]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []
  for k, e in zip(kor, eng):
    src_line = e
    tar_line = k  

    src_line = [w for w in src_line.split()]
    tar_line_in = [w for w in ("<sos> " + tar_line).split()]  # teacher forcing을 위한 정답셋

    tar_line_out = [w for w in (tar_line + " <eos>").split()]  # sos와 eos
    
    encoder_input.append(src_line)
    decoder_input.append(tar_line_in)
    decoder_target.append(tar_line_out)
    
  return encoder_input, decoder_input, decoder_target

In [12]:
# 전처리 테스트
en_sent = "First of all, I vacuumed the floor."
kor_sent = "나는 먼저 청소기로 바닥을 밀었어요."

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 한국어 문장 :', kor_sent)
print('전처리 후 한국어 문장 :', preprocess_sentence(kor_sent))

전처리 전 영어 문장 : First of all, I vacuumed the floor.
전처리 후 영어 문장 : First of all I vacuumed the floor .
전처리 전 한국어 문장 : 나는 먼저 청소기로 바닥을 밀었어요.
전처리 후 한국어 문장 : 나는 먼저 청소기로 바닥을 밀었어요 .


### 전체 데이터 특성추출

1. 전체데이터 특성
2. 인코더 입력 크기 : 47
3. 디코더 입력 크기 : 31
4. 영어 단어 집합의 크기 : 79177
5. 한국어 단어 집합의 크기 213044

In [13]:
sents_en_in, sents_kor_in, sents_kor_out = load_preprocessed_data()

In [14]:
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_kor_in[:5])
print('디코더의 레이블 :',sents_kor_out[:5])

인코더의 입력 : [['Bible', "Coloring'", 'is', 'a', 'coloring', 'application', 'that', 'allows', 'you', 'to', 'experience', 'beautiful', 'stories', 'in', 'the', 'Bible.'], ['Do', 'you', 'work', 'at', 'a', 'City', 'bank?'], ["PURITO's", 'bestseller,', 'which', 'recorded', '4th', 'rough', '-cuts', 'by', 'words', 'of', 'mouth', 'from', 'abroad.'], ['In', 'Chapter', '11', 'Jesus', 'called', 'Lazarus', 'from', 'the', 'tomb', 'and', 'raised', 'him', 'from', 'the', 'dead.'], ['I', 'would', 'feel', 'grateful', 'to', 'know', 'how', 'many', 'stocks', 'will', 'be', 'secured', 'of', 'size', '6.5,', '7,', 'and', '8.']]
디코더의 입력 : [['<sos>', "'Bible", "Coloring'은", '성경의', '아름다운', '이야기를', '체험', '할', '수', '있는', '컬러링', '앱입니다.'], ['<sos>', '씨티은행에서', '일하세요?'], ['<sos>', '푸리토의', '베스트셀러는', '해외에서', '입소문만으로', '4차', '완판을', '기록하였다.'], ['<sos>', '11장에서는', '예수님이', '이번엔', '나사로를', '무덤에서', '불러내어', '죽은', '자', '가운데서', '살리셨습니다.'], ['<sos>', '6.5,', '7,', '8', '사이즈가', '몇', '개나', '더', '재입고', '될지', '제게', '알려주시면', '감사하겠습니다.']]
디코

In [15]:
# 영어
tokenizer_en = Tokenizer(filters="", lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

In [16]:
encoder_input

[[8163, 57633, 6, 4, 12153, 911, 11, 2122, 5, 3, 360, 364, 2263, 9, 1, 10738],
 [102, 5, 91, 27, 4, 2720, 8927],
 [23362, 57634, 85, 3173, 2879, 3466, 57635, 36, 730, 7, 2628, 32, 2123],
 [124, 10739, 2186, 3096, 292, 57636, 32, 1, 16344, 8, 2418, 217, 32, 1, 6140],
 [2,
  57,
  136,
  1918,
  3,
  55,
  116,
  92,
  5546,
  19,
  18,
  10449,
  7,
  524,
  57637,
  11365,
  8,
  9411],
 [16345,
  29206,
  19738,
  18424,
  8,
  791,
  10,
  6328,
  751,
  7,
  6141,
  85,
  23,
  622,
  22,
  111],
 [185, 107, 77, 5, 4, 276, 7, 28, 8, 15, 5390],
 [9412,
  7,
  4101,
  16,
  166,
  10,
  1,
  3097,
  6242,
  8,
  14,
  6,
  1738,
  288,
  14789,
  10,
  309],
 [2,
  136,
  203,
  43,
  75,
  221,
  28,
  29207,
  44,
  79,
  16,
  214,
  4,
  4783,
  495,
  20,
  246,
  11,
  2,
  46,
  63,
  5465],
 [943, 4, 181, 62, 125, 4, 810, 623, 1135, 938, 39, 3686, 7, 11, 810, 1697],
 [2, 310, 15, 196, 11, 2, 19, 114, 5, 1, 57638, 295, 25, 4, 2351],
 [185, 1, 483, 7, 1, 542, 16, 15, 452],
 [2, 

In [17]:
# 한국어

tokenizer_kor = Tokenizer(filters="", lower = False)
tokenizer_kor.fit_on_texts(sents_kor_in)
tokenizer_kor.fit_on_texts(sents_kor_out)

decoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)
decoder_target = tokenizer_kor.texts_to_sequences(sents_kor_out) 

각 문장을 입력층의 최대 길이만큼 패딩(padding)을 수행합니다. 

In [18]:
encoder_input = pad_sequences(encoder_input, maxlen=30,padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=25, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=25, padding='post')

In [19]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)
# 100,000~200,000 : 인코더 30 , 디코더 48
# 0~ 100,000 인코더 : 25 , 디코더 40

인코더의 입력의 크기(shape) : (400000, 30)
디코더의 입력의 크기(shape) : (400000, 25)
디코더의 레이블의 크기(shape) : (400000, 25)


In [20]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_kor.word_index) + 1

print(f"영어 단어 집합의 크기 : {src_vocab_size}, 한국어 단어 집합의 크기 : {tar_vocab_size}")
# 0 ~ 100,000 : 영어 단어 집합의 크기 : 127507, 프랑스어 단어 집합의 크기 : 51454
# 100,000 ~ 200,000 : 영어 단어 집합의 크기 : 140413, 프랑스어 단어 집합의 크기 : 56283
# 강제로 줄여서 학습하면 loss값 계산이안됨

영어 단어 집합의 크기 : 124034, 한국어 단어 집합의 크기 : 349862


In [21]:
src_to_index = tokenizer_en.word_index  # word : idx
index_to_src = tokenizer_en.index_word  # idx : word
tar_to_index = tokenizer_kor.word_index # word : idx
index_to_tar = tokenizer_kor.index_word # idx : word

In [22]:
indices = np.arange(encoder_input.shape[0]) # 문장 개수 만큼의 index 만들어줌.
np.random.shuffle(indices) # 섞을 거임.
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [392424 390204 316860 ... 281333 249535 254151]


In [23]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [24]:
decoder_input[indices[0]], decoder_target[indices[0]]  # 2:sos, 3:eos 
                                                       # 2와 3을 제외한 나머지가 같이야함. 다르면 잘못한 것. 

(array([     1,   4464,   2215, 222239,   1709,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0], dtype=int32),
 array([  4464,   2215, 222239,   1709,      2,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0], dtype=int32))

10%의 데이터를 test데이터로 분리하겠습니다. 

In [25]:
n_of_val = int(num_samples*0.1) # 0.1만 쓰겠다. 본인 설정.

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

이제 번역기를 빌드해보겠습니다. 

In [26]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Bidirectional, Concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

In [27]:
# 인코더
encoder_inputs = Input(shape = (None, ))

# 임베딩 층
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)

# 1층 LSTM Input 들어오는층 legacy
encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c =  encoder_lstm(enc_emb)

encoder_states = [state_h, state_c]

In [28]:
from keras.layers import AdditiveAttention 

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
      super().__init__()

      self.W1 = tf.keras.layers.Dense(units, use_bias = False) # W1@ht 
      self.W2 = tf.keras.layers.Dense(units, use_bias = False) # W2@hs 

      self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, value):

      # W1 @ ht 
      w1_query = self.W1(query) # 디코더 t시점의 은닉상태

      # W2 @ hs 
      w2_key = self.W2(value)

      # attention
      context_vector, attention_weights = self.attention(inputs = [w1_query, value, w2_key], return_attention_scores = True)

      return context_vector, attention_weights


In [29]:
# 디코더
decoder_inputs = Input(shape=(None,))

# 임베딩 층
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)

# 임베딩 결과
dec_emb = dec_emb_layer(decoder_inputs) 

######################
# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 
######################

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _ , _ = decoder_lstm(dec_emb,initial_state=encoder_states)

######################
# attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1)
# tf.newaxis : 축추가

attention = BahdanauAttention(hidden_units)
context_vector, _ = attention(S_, encoder_outputs)

concat = tf.concat([decoder_outputs, context_vector], axis=-1)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(concat)

######################

In [30]:
# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 64)     7938176     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 128)    44782336    ['input_2[0][0]']                
                                                                                              

In [31]:
# model.load_weights('/content/drive/MyDrive/음성인식/model_weights(Attention파이널).h5')

In [33]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, 
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),batch_size=128, epochs=5)

Epoch 1/5


ResourceExhaustedError: ignored

해볼수 있는것
1. LSTM 층쌓기 : 층을 deeper going deeper
2. Encoder 반대로 집어넣기 Decoder에 가까울수록 데이터 정보저장잘함
3. 10만에서 20만 학습(O) -> : 실제 데이터셋이 많으니 정확도 올라감
4. Dropout 적용하기 , 기본 dropout과 recurrent_dropout이존재
5. 양방향 RNN 사용 원리 : 언어 이해시 단어 순서도 중요하지만 결정적이지않음

In [ ]:
# model.save_weights('/content/drive/MyDrive/음성인식/model_weights(Attention파이널).h5')

## BLEU Score (Bilingual Evaluation Understudy Score)
기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법 중 하나인 BLEU(Bilingual Evaluation Understudy)를 측정해봅시다. 
- 논문 : BLEU: a Method for Automatic Evaluation of Machine Translation
- BLEU 점수는 기계 번역된 텍스트와 고품질 참조 번역 세트의 유사성을 측정하는 0과 1 사이의 숫자입니다. 
    - 값이 0이면 기계 번역된 출력이 참조 번역과 겹치는 부분이 없는 것을 의미하고(저품질)
    - 1이면 참조 번역과 완벽하게 겹치는 것을 의미합니다(고품질)

- BLEU 설명 
    - https://jrc-park.tistory.com/273
    - https://wikidocs.net/31695
    - https://cloud.google.com/translate/automl/docs/evaluate?hl=ko#bleu

In [ ]:
# 인코더 - 훈련과 동일 

encoder_model = Model(encoder_inputs,[encoder_outputs, encoder_states])

In [ ]:
# 디코더 
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

######################
encoder_state_h = Input(shape=(hidden_units,)) # 마지막 state
encoder_outputs2 = Input(shape =(None, hidden_units,)) # 전체시점 hidden state 가져옴
######################

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h, state_c = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs) # query
decoder_states2 = [state_h, state_c]

######################
S_ = tf.concat([encoder_state_h[:, tf.newaxis, :], decoder_outputs2[:, :-1, :]], axis=1)

# Attention
context_vector, att_score = attention(S_, encoder_outputs2) # key
decoder_concat = tf.concat([decoder_outputs2, context_vector], axis=-1) # 그전에는 input이 2개엿는데 이번엔 context_vector로 input 3개로 진행

# 모든 시점에 대해서 단어 예측 (Fully Connected)
decoder_outputs2 = decoder_dense(decoder_concat)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs, encoder_state_h, encoder_outputs2] + decoder_states_inputs,
    [decoder_outputs2, att_score] + decoder_states2)

######################

In [ ]:
def decode_sequence(input_seq):
  # 문장 전처리를 위해 추가한 부분
  ##################
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  # max_src_len, max_tar_len  = 47, 32

  # attention_plot = np.zeros((max_tar_len, max_src_len))

  # sentence = preprocess_sentence(sentence)

  # inputs = [tokenizer_en.word_index[i] for i in sentence.split(' ')]
  # inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
  #                                                        maxlen=max_src_len,
  #                                                        padding='post')
  # inputs = tf.convert_to_tensor(inputs)

  #################

  encoder_test, states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, att_score, h, c = decoder_model.predict([target_seq, states_value[0], encoder_test] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]
    
    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if sampled_char == '<eos>':
      break
    
    if len(decoded_sentence) >=150:
      stop_condition = True
    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char
    

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<eos>']):
      if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
        sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [ ]:
from pylab import *
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
    # you need to change the fname based on your system, and the Chinese can be displayed in the plot
    # font = FontProperties(fname=r"./data/TaipeiSansTCBeta-Regular.ttf", size=14)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    # set the x-tick/y-tick labels with list of string labels
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation = 90)
    ax.set_yticklabels(predicted_sentence, fontdict=fontdict)

    # set tick locators
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    plt.show()

In [ ]:
def translate(sentence):
  sentence, result, attention_plot = decode_sequence(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' '))-2, :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
for seq_index in range(100):
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:])
  print("-"*50)

In [ ]:
import numpy as np
from collections import Counter
from nltk import ngrams

In [ ]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action which ensures that the military commands of the party always obeys the ',          
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

In [ ]:
ref_train = [seq_to_tar(sen) for sen in decoder_input_train[0:100]]

In [ ]:
from tqdm.notebook import tqdm 

pred_train = []
for idx in tqdm(range(len(encoder_input_train[0:100]))):
    pred_train.append(decode_sequence(encoder_input_train[idx:idx+1]))

In [ ]:
bleu_score = 0
count = 0
for ref, pred in zip(ref_train, pred_train):  
  if len(ref) == 0 or len(pred) == 0:
      continue
  bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_train)
bleu_score

In [ ]:
ref_test = [seq_to_tar(sen) for sen in decoder_input_test[:100]]
pred_test = [decode_sequence(encoder_input_test[idx:idx+1]) for idx in tqdm(range(len(encoder_input_test[:100])))]

bleu_score = 0
for ref, pred in zip(ref_test, pred_test):
    if len(ref) == 0 or len(pred) == 0:
        continue
    bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_test)
bleu_score

### Reference

- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://wikidocs.net/86900